In [3]:
import pandas as pd
import polars as pl
import numpy as np

In [4]:
clickstream = pd.read_parquet("data/interactions.parquet")

In [5]:
# В качестве точки разбиения используйте 16 декабря 2022 года
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2022-12-16").date()
clickstream["started_at"] = pd.to_datetime(clickstream["started_at"]).dt.date
train_test_global_time_split_idx = clickstream["started_at"] < train_test_global_time_split_date
events_train = clickstream[train_test_global_time_split_idx]
events_test = clickstream[~train_test_global_time_split_idx]

In [6]:
del clickstream

In [5]:
als_scores = pd.read_parquet("data/als_scores.parquet")
enhanced_events = pd.read_parquet('data\enchanced_events.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Admin\AppData\Local\Temp\ipykernel_10600\2923317524.py:2: SyntaxWarning: invalid escape sequence '\e'
  enhanced_events = pd.read_parquet('data\enchanced_events.parquet')


In [5]:
import gc
gc.collect()

17

In [7]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2022-12-22").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy() 

In [8]:
als_recommendations = pl.from_pandas(als_scores)
enhanced_events = pl.from_pandas(enhanced_events)
candidates = (
    als_recommendations.select(["user_id", "track_id", "score"])
    .with_columns(pl.col("score").alias("als_score"))
    .join(
        enhanced_events.select(["user_id", "track_id", "user_genre_affinity", "user_artist_ratio"]),
        on=["user_id", "track_id"],
        how="outer"
    )
)

candidates

C:\Users\Admin\AppData\Local\Temp\ipykernel_10600\1660291312.py:6: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(


user_id,track_id,score,als_score,user_id_right,track_id_right,user_genre_affinity,user_artist_ratio
i64,i64,f64,f64,i32,i32,f64,f64
null,null,null,null,0,99262,0.076923,0.038462
null,null,null,null,0,589498,0.192308,0.153846
null,null,null,null,0,590262,0.192308,0.153846
null,null,null,null,0,590303,0.192308,0.153846
null,null,null,null,0,590692,0.192308,0.153846
…,…,…,…,…,…,…,…
225558,10277791,0.086993,0.086993,null,null,null,null
945575,27051026,0.015018,0.015018,null,null,null,null
189166,87659099,0.124514,0.124514,null,null,null,null


In [9]:
# Заполняем пропуски в столбце 'user_id' соответствующими значениями из 'user_id_right'
candidates = candidates.with_columns(pl.col('user_id').fill_null(pl.col('user_id_right')).alias('user_id'))

# Заполняем пропуски в столбце 'track_id' соответствующими значениями из 'track_id_right'
candidates = candidates.with_columns(pl.col('track_id').fill_null(pl.col('track_id_right')).alias('track_id'))

candidates = candidates.select(['user_id',	'track_id',	'als_score',	'user_genre_affinity',	'user_artist_ratio'])
candidates.write_parquet('data/candidates.parquet')

In [ ]:
candidates = pl.read_parquet('data/candidates.parquet')

In [11]:
events_labels = pl.from_pandas(events_labels)
# Add the target column to events_labels with a value of 1
events_labels = events_labels.with_columns(pl.lit(1).alias("target"))

# Perform an outer join on user_id and track_id
candidates = (
    events_labels.select(["user_id", "track_id", "target"])
    .join(candidates, 
    on=["user_id", "track_id"], 
    how="outer")
)

# Fill null values in the target column with 0 and convert to integer
candidates = candidates.with_columns(
    pl.col("target").fill_null(0).cast(pl.Int32)
)

# Sample 50 rows from the candidates DataFrame
print(candidates.sample(50))

C:\Users\Admin\AppData\Local\Temp\ipykernel_10600\3704159602.py:8: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(candidates,


shape: (50, 8)
┌─────────┬──────────┬────────┬──────────────┬─────────────┬───────────┬─────────────┬─────────────┐
│ user_id ┆ track_id ┆ target ┆ user_id_righ ┆ track_id_ri ┆ als_score ┆ user_genre_ ┆ user_artist │
│ ---     ┆ ---      ┆ ---    ┆ t            ┆ ght         ┆ ---       ┆ affinity    ┆ _ratio      │
│ i32     ┆ i32      ┆ i32    ┆ ---          ┆ ---         ┆ f64       ┆ ---         ┆ ---         │
│         ┆          ┆        ┆ i64          ┆ i64         ┆           ┆ f64         ┆ f64         │
╞═════════╪══════════╪════════╪══════════════╪═════════════╪═══════════╪═════════════╪═════════════╡
│ null    ┆ null     ┆ 0      ┆ 435969       ┆ 68562711    ┆ null      ┆ 0.619048    ┆ 0.006667    │
│ null    ┆ null     ┆ 0      ┆ 1084605      ┆ 73122673    ┆ null      ┆ 0.344086    ┆ 0.001195    │
│ null    ┆ null     ┆ 0      ┆ 746156       ┆ 25301458    ┆ 0.003343  ┆ null        ┆ null        │
│ null    ┆ null     ┆ 0      ┆ 716553       ┆ 1695521     ┆ 0.079709  ┆ nul

In [12]:

candidates = candidates.with_columns(pl.col('user_id').fill_null(pl.col('user_id_right')).alias('user_id'))

# Заполняем пропуски в столбце 'track_id' соответствующими значениями из 'track_id_right'
candidates = candidates.with_columns(pl.col('track_id').fill_null(pl.col('track_id_right')).alias('track_id'))
candidates = candidates.select(['user_id',	'track_id',	'als_score',	'user_genre_affinity',	'user_artist_ratio','target'])
candidates.write_parquet('data/candidates_target.parquet')

In [7]:
candidates = pl.read_parquet('data/candidates_target.parquet')

In [9]:
# Выбираем кандидатов, у которых хотя бы в одной строке 'target' равно 1
candidates_with_target_1 = candidates.filter(pl.col("target") == 1)
user_id_list = list(candidates_with_target_1.select("user_id").unique())[0]
selected_candidates = candidates.filter(pl.col("user_id").is_in(user_id_list))
# Выбираем ещё 4 примера, где 'target' равно 0 для каждого 'user_id'
candidates_with_target_0 = selected_candidates.filter(pl.col("target") == 0).group_by("user_id").head(4)

# Объединяем выбранные кандидаты
filtered_candidates = pl.concat([candidates_with_target_1, candidates_with_target_0])

print(filtered_candidates)


shape: (8_946_262, 6)
┌─────────┬──────────┬───────────┬─────────────────────┬───────────────────┬────────┐
│ user_id ┆ track_id ┆ als_score ┆ user_genre_affinity ┆ user_artist_ratio ┆ target │
│ ---     ┆ ---      ┆ ---       ┆ ---                 ┆ ---               ┆ ---    │
│ i64     ┆ i64      ┆ f64       ┆ f64                 ┆ f64               ┆ i32    │
╞═════════╪══════════╪═══════════╪═════════════════════╪═══════════════════╪════════╡
│ 69778   ┆ 58609142 ┆ 0.549803  ┆ null                ┆ null              ┆ 1      │
│ 106629  ┆ 29456722 ┆ 0.075868  ┆ null                ┆ null              ┆ 1      │
│ 1363837 ┆ 66726338 ┆ 0.101642  ┆ null                ┆ null              ┆ 1      │
│ 1160082 ┆ 4447862  ┆ 0.012785  ┆ null                ┆ null              ┆ 1      │
│ 757986  ┆ 25727188 ┆ 0.220306  ┆ null                ┆ null              ┆ 1      │
│ …       ┆ …        ┆ …         ┆ …                   ┆ …                 ┆ …      │
│ 974311  ┆ 10241    ┆ null     

In [31]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.4 MB 1.3 MB/s eta 0:01:20
   ---------------------------------------- 0.1/102.4 MB 787.7 kB/s eta 0:02:10
   ---------------------------------------- 0.1/102.4 MB 1.1 MB/s eta 0:01:37
   ---------------------------------------- 0.3/102.4 MB 1.6 MB/s eta 0:01:04
   ---------------------------------------- 0.5/102.4 MB 2.2 MB/s eta 0:00:47
   ---------------------------------------- 0.7/102.4 MB 2.4 MB/s eta 0:00:43
   ---------------------------------------- 0.8/102.4 MB 2.4 MB/s eta 0:00:42
   ---------------------------------------- 1.3/102.4 MB 3.3 MB/s eta 0:00:31
    --------------------------------------- 1.7/102.4 MB 3.9 MB/s eta 0:00:26
    --------------------------------------- 1.7/102.4 MB 3.9 MB/s eta 0:00:26
    --------------------------------------- 1.7/102.4 MB 3.3 MB/s eta 0:00

In [2]:
from catboost import CatBoostClassifier, Pool

In [10]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'user_genre_affinity', 'user_artist_ratio']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=np.array(filtered_candidates[features]), 
    label=np.array(filtered_candidates[target])
)

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=10,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data) 

0:	learn: 0.5980682	total: 338ms	remaining: 33.4s
10:	learn: 0.1787278	total: 2.64s	remaining: 21.3s
20:	learn: 0.0631459	total: 4.94s	remaining: 18.6s
30:	learn: 0.0238807	total: 7.18s	remaining: 16s
40:	learn: 0.0098027	total: 9.36s	remaining: 13.5s
50:	learn: 0.0047476	total: 11.5s	remaining: 11.1s
60:	learn: 0.0028649	total: 13.7s	remaining: 8.77s
70:	learn: 0.0021552	total: 15.9s	remaining: 6.49s
80:	learn: 0.0018894	total: 18.1s	remaining: 4.23s
90:	learn: 0.0017891	total: 20.2s	remaining: 2s
99:	learn: 0.0017605	total: 22.2s	remaining: 0us


In [11]:
cb_model.save_model('catboost_model.cbm')

In [3]:
cb_model = CatBoostClassifier()
cb_model.load_model('catboost_model.cbm')

# Делаем предсказания


In [28]:
events_test_2 = pl.from_pandas(events_test_2)
# Add the target column to events_labels with a value of 1
events_test_2 = events_test_2.with_columns(pl.lit(1).alias("target"))

# Perform an outer join on user_id and track_id
candidates2 = (
    events_test_2.select(["user_id", "track_id", "target"])
    .join(candidates, 
    on=["user_id", "track_id"], 
    how="outer")
)

# Fill null values in the target column with 0 and convert to integer
candidates2 = candidates2.with_columns(
    pl.col("target").fill_null(0).cast(pl.Int32)
)

# Sample 50 rows from the candidates DataFrame
print(candidates2.sample(50))

C:\Users\Admin\AppData\Local\Temp\ipykernel_19548\552681065.py:8: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(candidates,


shape: (50, 9)
┌─────────┬──────────┬────────┬─────────────┬───┬───────────┬────────────┬────────────┬────────────┐
│ user_id ┆ track_id ┆ target ┆ user_id_rig ┆ … ┆ als_score ┆ user_genre ┆ user_artis ┆ target_rig │
│ ---     ┆ ---      ┆ ---    ┆ ht          ┆   ┆ ---       ┆ _affinity  ┆ t_ratio    ┆ ht         │
│ i32     ┆ i32      ┆ i32    ┆ ---         ┆   ┆ f64       ┆ ---        ┆ ---        ┆ ---        │
│         ┆          ┆        ┆ i64         ┆   ┆           ┆ f64        ┆ f64        ┆ i32        │
╞═════════╪══════════╪════════╪═════════════╪═══╪═══════════╪════════════╪════════════╪════════════╡
│ null    ┆ null     ┆ 0      ┆ 639131      ┆ … ┆ 0.01272   ┆ null       ┆ null       ┆ 0          │
│ null    ┆ null     ┆ 0      ┆ 349693      ┆ … ┆ null      ┆ 0.005269   ┆ 0.005855   ┆ 0          │
│ null    ┆ null     ┆ 0      ┆ 1289728     ┆ … ┆ 0.131252  ┆ null       ┆ null       ┆ 0          │
│ null    ┆ null     ┆ 0      ┆ 554232      ┆ … ┆ null      ┆ 0.035578   ┆ 0

In [13]:
# Заполняем пропуски в столбце 'user_id' соответствующими значениями из 'user_id_right'
candidates2 = candidates2.with_columns(pl.col('user_id').fill_null(pl.col('user_id_right')).alias('user_id'))

# Заполняем пропуски в столбце 'track_id' соответствующими значениями из 'track_id_right'
candidates2 = candidates2.with_columns(pl.col('track_id').fill_null(pl.col('track_id_right')).alias('track_id'))

candidates2 = candidates2.select(['user_id',	'track_id',	'als_score',	'user_genre_affinity',	'user_artist_ratio','target'])
candidates2.write_parquet('data/candidates_to_rank.parquet')

In [4]:
candidates2 = pl.read_parquet('data/candidates_to_rank.parquet')

In [5]:
features = ['als_score', 'user_genre_affinity', 'user_artist_ratio']
target = 'target'
inference_data = Pool(data=np.array(candidates2[features]))
predictions = cb_model.predict_proba(inference_data)

candidates2 = candidates2.with_columns(pl.Series(predictions[:, 1]).alias('cb_score'))
candidates2

user_id,track_id,als_score,user_genre_affinity,user_artist_ratio,target,cb_score
i64,i64,f64,f64,f64,i32,f64
0,99262,null,0.076923,0.038462,0,0.000027
0,589498,null,0.192308,0.153846,0,0.000028
0,590262,null,0.192308,0.153846,0,0.000028
0,590303,null,0.192308,0.153846,0,0.000028
0,590692,null,0.192308,0.153846,0,0.000028
…,…,…,…,…,…,…
527,18862,null,null,null,1,0.999954
525255,85652278,null,null,null,1,0.999954
32303,37079539,null,null,null,1,0.999954


In [6]:
del predictions, inference_data
# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates2.sort(by=["user_id", "cb_score"], descending=[False, True])
del candidates2
candidates_to_rank

user_id,track_id,als_score,user_genre_affinity,user_artist_ratio,target,cb_score
i64,i64,f64,f64,f64,i32,f64
0,589498,null,0.192308,0.153846,0,0.000028
0,590262,null,0.192308,0.153846,0,0.000028
0,590303,null,0.192308,0.153846,0,0.000028
0,590692,null,0.192308,0.153846,0,0.000028
0,10327926,null,0.038462,0.115385,0,0.000027
…,…,…,…,…,…,…
1374582,54559796,null,0.285024,0.028986,0,0.000027
1374582,54559802,null,0.285024,0.028986,0,0.000027
1374582,72875547,null,0.285024,0.028986,0,0.000027


In [7]:
# Rank the cb_score within each group of user_id
candidates_to_rank = candidates_to_rank.with_columns(
    pl.col('cb_score').rank(method='dense', descending=True).over('user_id').alias('rank')
)

candidates_to_rank

user_id,track_id,als_score,user_genre_affinity,user_artist_ratio,target,cb_score,rank
i64,i64,f64,f64,f64,i32,f64,u32
0,589498,null,0.192308,0.153846,0,0.000028,1
0,590262,null,0.192308,0.153846,0,0.000028,1
0,590303,null,0.192308,0.153846,0,0.000028,1
0,590692,null,0.192308,0.153846,0,0.000028,1
0,10327926,null,0.038462,0.115385,0,0.000027,2
…,…,…,…,…,…,…,…
1374582,54559796,null,0.285024,0.028986,0,0.000027,36
1374582,54559802,null,0.285024,0.028986,0,0.000027,36
1374582,72875547,null,0.285024,0.028986,0,0.000027,36


In [10]:
# Add a row number within each group of user_id, sorted by rank in descending order
candidates_to_rank = candidates_to_rank.with_columns(
    pl.col('rank').rank(method='ordinal', descending=True).over('user_id').alias('row_num')
)

# Filter to keep only the top 100 rows per user_id
candidates_to_rank = candidates_to_rank.filter(pl.col('row_num') <= 100)

In [12]:
candidates_to_rank.sort(by=["user_id", "row_num"], descending=[False, False])

user_id,track_id,als_score,user_genre_affinity,user_artist_ratio,target,cb_score,rank,row_num
i64,i64,f64,f64,f64,i32,f64,u32,u32
0,17802673,null,0.115385,0.038462,0,0.000027,12,1
0,17802674,null,0.115385,0.038462,0,0.000027,12,2
0,99262,null,0.076923,0.038462,0,0.000027,11,3
0,14370719,null,0.038462,0.038462,0,0.000027,10,4
0,18042047,null,0.038462,0.038462,0,0.000027,10,5
…,…,…,…,…,…,…,…,…
1374582,72725485,null,0.125604,0.004831,0,0.000038,23,96
1374582,62405492,null,0.125604,0.004831,0,0.000038,23,97
1374582,48592372,null,0.125604,0.004831,0,0.000038,23,98


In [14]:
candidates_to_rank.write_parquet('data/cb_predictions.parquet')

In [8]:
candidates_to_rank = pl.read_parquet('data/cb_predictions.parquet')

In [46]:
import polars as pl
from collections import defaultdict
import numpy as np
from tqdm import tqdm  # <-- Добавляем импорт

def evaluate_recommendations(recommendations_df: pl.DataFrame, validation_df: pl.DataFrame, k: int):
    """Оценивает качество рекомендаций с прогресс-барами."""
    # Инициализация переменных для метрик
    total_relevant = 0
    total_recommended = 0
    total_relevant_and_recommended = 0
    all_items = set()
    recommended_items = set()
    item_popularity = defaultdict(int)

    # Прогресс-бар для подсчёта популярности
    print("[1/4] Подсчёт популярности треков:")
    for track_id in tqdm(validation_df['track_id'], desc="Обработка треков"):
        item_popularity[track_id] += 1

    # Группировка с сортировкой по score
    print("\n[2/4] Группировка данных:")
    with tqdm(total=3, desc="Подготовка данных") as pbar:
        recommendations_grouped_df = recommendations_df.sort('score', descending=True).group_by('user_id').agg([
            pl.col('track_id').head(k).alias('track_id_list')
        ])
        pbar.update(1)
        
        validation_grouped_df = validation_df.group_by('user_id').agg([
            pl.col('track_id').implode().alias('track_id_list')
        ])
        pbar.update(1)
        
        recommendations_grouped = recommendations_grouped_df.to_dict(as_series=False)
        validation_grouped = validation_grouped_df.to_dict(as_series=False)
        validation_user_ids = set(validation_grouped['user_id'])
        pbar.update(1)

    # Обработка пользователей с прогресс-баром
    print("\n[3/4] Оценка рекомендаций:")
    user_ids = recommendations_grouped['user_id']
    total_users = len(user_ids)
    
    for idx, (user_id, recommended_tracks) in tqdm(
        enumerate(zip(user_ids, recommendations_grouped['track_id_list'])),
        total=total_users,
        desc="Обработка пользователей"
    ):
        if user_id not in validation_user_ids:
            continue
            
        real_tracks = validation_grouped['track_id_list'][validation_grouped['user_id'].index(user_id)]
        real_tracks_set = set(real_tracks)
        
        relevant_count = len(real_tracks_set)
        recommended_count = len(recommended_tracks)
        relevant_and_recommended = len(real_tracks_set & set(recommended_tracks))
        
        total_relevant += relevant_count
        total_recommended += recommended_count
        total_relevant_and_recommended += relevant_and_recommended
        
        for track_id in recommended_tracks:
            recommended_items.add(track_id)
            all_items.add(track_id)
            item_popularity[track_id] += 0

    # Расчёт novelty с прогресс-баром
    print("\n[4/4] Расчёт Novelty:")
    novelty_scores = []
    user_ids = recommendations_grouped['user_id']
    
    for user_id, recommended_tracks in tqdm(
        zip(user_ids, recommendations_grouped['track_id_list']),
        total=len(user_ids),
        desc="Обработка Novelty"
    ):
        if user_id not in validation_user_ids:
            continue
            
        for track_id in recommended_tracks:
            popularity = item_popularity.get(track_id, 1)
            try:
                novelty_scores.append(1.0 / popularity)
            except ZeroDivisionError:
                novelty_scores.append(0)

    # Вычисление финальных метрик
    metrics = {
        'recall': total_relevant_and_recommended / total_relevant if total_relevant > 0 else 0,
        'precision': total_relevant_and_recommended / total_recommended if total_recommended > 0 else 0,
        'coverage': len(recommended_items) / len(all_items) if len(all_items) > 0 else 0,
        'novelty': np.mean(novelty_scores) if novelty_scores else 0
    }
    
    print("\n✅ Оценка завершена")
    return metrics


In [47]:
k = 5  # Оцениваем топ-5 рекомендации
recommendations = candidates_to_rank.select(['user_id', 'track_id','cb_score']).rename({
    'cb_score': 'score'
})

validation = pl.from_pandas(events_test_2[['user_id', 'track_id']])
# Вычисление метрик
metrics = evaluate_recommendations(recommendations, validation, k)
print(metrics)

[1/4] Подсчёт популярности треков:


Обработка треков: 100%|██████████| 7522628/7522628 [00:02<00:00, 3451229.82it/s]



[2/4] Группировка данных:


Подготовка данных: 100%|██████████| 3/3 [00:06<00:00,  2.02s/it]



[3/4] Оценка рекомендаций:


Обработка пользователей: 100%|██████████| 1373221/1373221 [30:36<00:00, 747.56it/s]



[4/4] Расчёт Novelty:


Обработка Novelty: 100%|██████████| 1373221/1373221 [00:01<00:00, 1305329.49it/s]



✅ Оценка завершена
{'recall': 0.01802308980319112, 'precision': 0.042548871371109846, 'coverage': 1.0, 'novelty': np.float64(0.04572598680934325)}


In [ ]:
pops = pl.read_parquet('data/top_popular.parquet')


user_id,recommended_track_id
i64,i64
3,68348389
3,72055319
3,48591706
3,37077421
3,31190258
…,…
1374582,25301457
1374582,59930542
1374582,76957294


In [49]:
pops = pops.with_columns(pl.lit(1).alias("score"))
pops

user_id,recommended_track_id,score
i64,i64,i32
3,68348389,1
3,72055319,1
3,48591706,1
3,37077421,1
3,31190258,1
…,…,…
1374582,25301457,1
1374582,59930542,1
1374582,76957294,1


In [51]:
pops =pops.select(['user_id', 'recommended_track_id','score']).rename({
    'recommended_track_id': 'track_id'
})
metrics = evaluate_recommendations(pops, validation, k)
print(metrics)

[1/4] Подсчёт популярности треков:


Обработка треков: 100%|██████████| 7522628/7522628 [00:02<00:00, 3419797.67it/s]



[2/4] Группировка данных:


Подготовка данных: 100%|██████████| 3/3 [00:00<00:00,  3.55it/s]



[3/4] Оценка рекомендаций:


Обработка пользователей: 100%|██████████| 783525/783525 [28:17<00:00, 461.63it/s] 



[4/4] Расчёт Novelty:


Обработка Novelty: 100%|██████████| 783525/783525 [00:00<00:00, 1164230.06it/s]



✅ Оценка завершена
{'recall': 0.0017804947951699857, 'precision': 0.004202955624834278, 'coverage': 1.0, 'novelty': np.float64(0.005233152182120533)}


In [55]:
import boto3
import os

class Config:
    _config_loaded = False

    @classmethod
    def load_config(cls, filename='config.txt'):
        """Загружает конфигурацию из файла"""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Config file {filename} not found")

        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    setattr(cls, key.strip(), value.strip())
        
        cls._config_loaded = True

    @classmethod
    def get(cls, key):
        if not cls._config_loaded:
            cls.load_config()
        return getattr(cls, key, None)

def get_session():
    # Загружаем конфиг при первом вызове
    if not Config._config_loaded:
        Config.load_config()
    
    return boto3.session.Session().client(
        service_name=Config.get('S3_SERVICE_NAME'),
        endpoint_url=Config.get('S3_ENDPOINT_URL'),
        aws_access_key_id=Config.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=Config.get('AWS_SECRET_ACCESS_KEY')
    )

def upload_files_to_s3(file_paths: list, s3_bucket: str, s3_prefix: str = 'recsys/data/'):
    """Загружает файлы в S3-бакет по указанному пути"""
    s3 = get_session()
    
    for local_path in file_paths:
        if not os.path.exists(local_path):
            raise FileNotFoundError(f"Файл {local_path} не найден")
            
        s3_key = os.path.join(s3_prefix, os.path.basename(local_path))
        
        with open(local_path, 'rb') as f:
            s3.upload_fileobj(
                Fileobj=f,
                Bucket=s3_bucket,
                Key=s3_key
            )
        print(f"Файл {local_path} успешно загружен в s3://{s3_bucket}/{s3_key}")


bucket_name = 's3-student-mle-20250130-833968fcc1'

files_to_upload = [
        'data\items.parquet',
        'data\events.parquet',
    ]

# Загрузка файлов
upload_files_to_s3(
        file_paths=files_to_upload,
        s3_bucket=bucket_name
    )


<>:62: SyntaxWarning: invalid escape sequence '\i'
<>:63: SyntaxWarning: invalid escape sequence '\e'
<>:62: SyntaxWarning: invalid escape sequence '\i'
<>:63: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Admin\AppData\Local\Temp\ipykernel_1308\1825838838.py:62: SyntaxWarning: invalid escape sequence '\i'
  'data\items.parquet',
C:\Users\Admin\AppData\Local\Temp\ipykernel_1308\1825838838.py:63: SyntaxWarning: invalid escape sequence '\e'
  'data\events.parquet',


Файл data\items.parquet успешно загружен в s3://s3-student-mle-20250130-833968fcc1/recsys/data/items.parquet
Файл data\events.parquet успешно загружен в s3://s3-student-mle-20250130-833968fcc1/recsys/data/events.parquet
